# Linear Integer Programming

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pulp

In [ ]:
import networkx as nx
import pulp
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from tqdm.notebook import tqdm as pb
import sys
import pylab 
pylab.rcParams['figure.figsize'] = (10, 10)

import time

## Data

In [ ]:
#Parser de fichiers
df = pd.read_csv("drive/My Drive/Colab Notebooks/data.txt",header=None)
M = df.values
df.head()

,0,1,2,3
0,0,-11.03,-82.33,46
1,1,-4.93,-98.05,96
2,2,81.52,2.41,49
3,3,36.82,-42.62,71
4,4,-69.86,11.88,27


## Object

In [ ]:
class LPSolver_AGAR():
    def __init__(self, M, knn=-1):
        self.data = M
        self.knn = knn
    
    def _create_distance_edges(self):
        dim = self.data.shape[0]
        edges_dist = {}
        edges_list = []
        for i in pb(range(dim)):
            temp = np.zeros((dim))
            for j in range(dim):
                temp[j] = math.sqrt((self.data[i,1] - self.data[j,1])**2 + (self.data[i,2] - self.data[j,2])**2)
                
            min_dist = np.argsort(temp)
            for j in range(1,min(self.knn+1, dim)):
                to = min_dist[j]
                edges_list.append((i, to))
                edges_list.append((to, i))
                edges_dist[i, to] = temp[to]
                edges_dist[to, i] = temp[to]
        
        # init coords
        temp = np.zeros((dim))
        for j in range(dim):
            temp[j] = math.sqrt((0. - self.data[j,1])**2 + (0. - self.data[j,2])**2)
                
        min_dist = np.argsort(temp)
        for j in range(1,min(self.knn+1, dim)):
            to = min_dist[j]
            edges_list.append((dim, to))
            edges_dist[dim, to] = temp[to]
                    
        return [edges_dist, edges_list, dim+1]

    def _get_solution_and_add_constraints(self, G, edge_activation, edge_distances, source, target):
        validated = []
        correct = []
        actual_node = source
        edges_choosen = [[] for i in range(len(G.nodes))]
        for i,j in G.edges:
            if edge_activation[i,j].value() == 1.0:
                edges_choosen[i].append(j)
        while actual_node != target:
            if len(edges_choosen[actual_node]) != 1:
                print(actual_node, edges_choosen[actual_node])
                print("error")
            else:
                correct.append(actual_node)
                next_ = edges_choosen[actual_node][0]
                validated.append((actual_node,next_))
                actual_node = next_
        correct.append(actual_node)
        # detect cycle and will delete one of the edges
        cycles = []
        memory = []
        for i in range(len(edges_choosen)):
            if i not in correct and i not in memory and len(edges_choosen[i]) != 0:
                temp = []
                final_node = i
                temp.append(final_node)
                final_node = edges_choosen[final_node][0]
                while final_node != i:
                    next_ = edges_choosen[final_node][0]
                    temp.append(final_node)
                    final_node = next_
                memory += temp
                cycles.append(temp)
        # select first edge of each cycles and will add it to the future constraint
        return cycles, validated

    def create_graph(self, edges_dist, edges_list):
        # Create the Graph
        G = nx.DiGraph()
        # add nodes
        for i in range(dim):
            G.add_node(i)
        # add edges
        G.add_edges_from(edges_list)
        nx.set_edge_attributes(G, edges_dist, 'weight')
        """
        pos = nx.spring_layout(G)
        nx.draw_networkx(G, pos)
        plt.show()
        """
        return G

    def create_stable_problem(self, G, max_pair, dim, _max_time, init_rewards):
        time1 = time.time()

        rewards = np.append(init_rewards, 0)
        # Source and target
        source = max_pair[0]
        target = max_pair[1]
            
        # Non changing parameters
        edge_distances = nx.get_edge_attributes(G, 'weight')

        # instantiate
        prob = pulp.LpProblem("MaxRewardInLimitedTime", pulp.LpMaximize)
        # binary variable to state a link is chosen or not
        edge_activation = {}

        vertices = [[] for i in range(dim)]
        vertices
        for i, j in G.edges.keys():
            x = pulp.LpVariable("activ(%s,%s)" % (i,j), cat=pulp.LpBinary)
            edge_activation[i, j] = x
            if i == source:
                vertices[i].append(edge_activation[i, j])
            else:
                vertices[j].append(edge_activation[i, j])
        
        vertices = [pulp.lpSum(vertices[i])*rewards[i] for i in range(dim)]
        # objective function
        prob += pulp.lpSum(vertices), "Total Hop Count"

        # constraints
        constraints = []
        for node in G.nodes:
            if node == source:
                prob += pulp.lpSum([edge_activation[i, k] for i, k in G.edges if k == node]) - \
                      pulp.lpSum([edge_activation[k, j] for k, j in G.edges if k == node]) == -1
                prob += pulp.lpSum([edge_activation[i, k] for i, k in G.edges if k == node]) == 0
            elif node == target:
                prob += pulp.lpSum([edge_activation[i, k] for i, k in G.edges if k == node]) - \
                      pulp.lpSum([edge_activation[k, j] for k, j in G.edges if k == node]) == 1
                prob += pulp.lpSum([edge_activation[k, j] for k, j in G.edges if k == node]) == 0
            else:   
                prob += pulp.lpSum([edge_activation[i, k] for i, k in G.edges if k == node]) - \
                      pulp.lpSum([edge_activation[k, j] for k, j in G.edges if k == node]) == 0
                prob += pulp.lpSum([edge_activation[i, k] for i, k in G.edges if k == node]) + \
                      pulp.lpSum([edge_activation[k, j] for k, j in G.edges if k == node]) <= 2
         
        # for each edge 
        for i, j in G.edges:
            try:
                edge_activation[j, i]
                prob += pulp.lpSum([edge_activation[i, j], edge_activation[j, i]]) <= 1
            except Exception:
                pass
     
        # inferior to max time
        prob += pulp.lpSum([edge_distances[i, j] * edge_activation[i, j] for i, j in G.edges]) <= _max_time

        print(time.time()-time1)

        return prob, edge_activation, edge_distances, rewards
        

    def _optimize(self, G, max_pair, prob, edge_activation, edge_distances, time_limit, cycles, rewards):
        source = max_pair[0]
        target = max_pair[1]

        # break previous cycles and hopefully will come over
        for cycle in cycles:
            vertices_nb = len(cycle)
            cycle_cons = [edge_activation[cycle[k-1], cycle[k]] for k in range(vertices_nb)]
            cycle_cons_inv = [edge_activation[cycle[k], cycle[k-1]] for k in range(vertices_nb)]

            prob += pulp.lpSum([cycle_cons]) <= vertices_nb-1
            prob += pulp.lpSum([cycle_cons_inv]) <= vertices_nb-1
                   
        
        # solve
        prob.solve(pulp.PULP_CBC_CMD(timeLimit=time_limit, msg=1, gapRel=0))

        if prob.status != 1:
            raise Exception("Problem can't be solved : {}".format(pulp.LpStatus[prob.status]))
        else:
            new_cycles, path = self._get_solution_and_add_constraints(G, edge_activation, edge_distances, source, target)
            return path, new_cycles, prob, pulp.value(prob.objective)

In [ ]:
def get_sub_section(M, coord1, coord2):
    indices = []
    for i in range(M.shape[0]):
        if M[i,1] >= coord1[0] and M[i,1] <= coord2[0] and M[i, 2] >= coord1[1] and M[i, 2] <= coord2[1]:
            indices.append(i)
    # colors
    colors = np.zeros((M.shape[0]))
    for i in indices:
        colors[i] = 1
    return np.copy(M)[indices,:], colors

In [ ]:
def create_dist_matrix(M):
    dim = M.shape[0]
    dist_M = np.zeros((dim,dim))
    for i in pb(range(dim)):
        for j in range(dim):
            dist_M[i,j] = math.sqrt((M[i,1] - M[j,1])**2 + (M[i,2] - M[j,2])**2)
    return dist_M

## Processing

In [ ]:
indices = M[:,3] > 30
M_threshold = M[indices]
M_threshold.shape[0]
distances_M = create_dist_matrix(M_threshold)

In [ ]:
lp_solver = LPSolver_AGAR(M_threshold, 3)

In [ ]:
[edges_dist, edges_list, dim] = lp_solver._create_distance_edges()

In [ ]:
G = lp_solver.create_graph(edges_dist, edges_list)

In [ ]:
prob, edge_activation, edge_distances, rewards = lp_solver.create_stable_problem(G.copy(), (dim-1,0), dim, 10000., M_threshold[:,3])

220.01911759376526


In [ ]:
path, update_constraints, prob, sum_rewards = lp_solver._optimize(G.copy(), (dim-1,0), prob, edge_activation, edge_distances, 60*10*6, [], rewards)

In [ ]:
while len(update_constraints) != 0:
    path, update_constraints, prob, sum_rewards = lp_solver._optimize(G.copy(), (dim-1,0), prob, edge_activation, edge_distances, 60*10, update_constraints, rewards)
    print(len(path), len(update_constraints), sum_rewards)

In [ ]:
print(len(path), len(update_constraints), sum_rewards)

One improvement could be to take in account that the system will take a lot of time to propose only one unique path. Then if the number of cycles is really low (max 10 for example) we coul aggregate the cycle to the path by selecting the most nearest point of path-cycle and create/delete edges to respect the path structure